In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
import re

#Function to return predicates in FOL statement
def predicates(st1):
    exp = '[a-z~]+\([A-Za-z,]+\)'
    return re.findall(exp, st1)

#Returns attributes
def attributes(st1):
    exp = '\([^)]+\)'
    match = re.findall(exp, st1)
    return [m for m in str(match) if m.isalpha()]

#Applies deMorgan's law to the statement
def deMorgan(sent):
    st1 = ''.join(list(sent).copy())
    st1 = st1.replace('~~','')
    flag = '[' in st1
    st1 = st1.replace('~[','')
    st1 = st1.strip(']')
    for p in predicates(st1):
        st1 = st1.replace(p, f'~{p}')
    s = list(st1)
    for i, c in enumerate(st1):
        if c == '|':
            s[i] = '&'
        elif c == '&':
            s[i] = '|'
    st1 = ''.join(s)    
    st1 = st1.replace('~~','')
    return f'[{st1}]' if flag else st1

In [23]:
#Function to perform skolemization using a set of constants to remove existential quantifier
def skolemize(sent):
    constants = [f'{chr(c)}' for c in range(ord('A'), ord('Z')+1)]
    statement = ''.join(list(sent).copy())
    match = re.findall('[∀∃].', statement)
    for m in match[::-1]:
        statement = statement.replace(m, '')
        statements = re.findall('\[\[[^]]+\]]', statement)
        for stt in statements:
            statement = statement.replace(stt, stt[1:-1])
        for p in predicates(statement):
            att = attributes(p)
            if ''.join(att).islower():
                statement = statement.replace(m[1], constants.pop(0))
            else:
                aU = [a for a in att if not a.islower()][0]
                statement = statement.replace(aU, f'{constants.pop(0)}({m[1]})')
    return statement

In [24]:
import re

def FOLtoCNF(fol):    
    statement = fol.replace("<=>", "_") #Removing Biconditionals
    while '_' in statement:
        idx = statement.index('_')
        st1 = '[' + statement[:idx] + '=>' + statement[idx+1:] + '] & ['+ statement[idx+1:] + '=>' + statement[:idx] + ']'
        statement = st1
    statement = statement.replace("=>", "-") #Removing implications
    exp = '\[([^]]+)\]'
    statements = re.findall(exp, statement)
    for i, s in enumerate(statements):
        if '[' in s and ']' not in s:
            statements[i] += ']'
    for s in statements:
        statement = statement.replace(s, FOLtoCNF(s))
    while '-' in statement: 
        idx = statement.index('-')
        b = statement.index('[') if '[' in statement else 0
        st1 = '~' + statement[b:idx] + '|' + statement[idx+1:]
        statement = statement[:b] + st1 if b > 0 else st1
    while '~∀' in statement:
        idx = statement.index('~∀')
        statement = list(statement)
        statement[idx], statement[idx+1], statement[idx+2] = '∃', statement[idx+2], '~'
        statement = ''.join(statement)
    while '~∃' in statement:
        idx = statement.index('~∃')
        s = list(statement)
        s[idx], s[idx+1], s[idx+2] = '∀', s[idx+2], '~'
        statement = ''.join(s)
    statement = statement.replace('~[∀','[~∀') #Moving the ~ inside the brackets
    statement = statement.replace('~[∃','[~∃')
    exp = '(~[∀|∃].)'
    statements = re.findall(exp, statement)
    for s in statements:
        statement = statement.replace(s, FOLtoCNF(s))
    exp = '~\[[^]]+\]'
    statements = re.findall(exp, statement)
    for s in statements:
        statement = statement.replace(s, deMorgan(s)) #Using deMorgan's law
    return statement

In [25]:
def main():
    print("Enter the staement in First Order Logic (FOL):")
    fol = input()
    print("The FOL statement in Conjunctive Normal Form (CNF) is as follows:")
    print(skolemize(FOLtoCNF(fol)))

In [26]:
#Statement in English: Every Human Being Has Brain
main()

Enter the staement in First Order Logic (FOL):
∀xHuman(x) => ∃yBrain(y) & Has(x,y)
The FOL statement in Conjunctive Normal Form (CNF) is as follows:
~Human(D) | Brain(E(x)) & Has(D,E(x))
